In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize , pos_tag, sent_tokenize
import warnings
from gensim import corpora , models , similarities
import jieba
import re

## Underthesea

In [2]:
data = ["Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond's White Beauty 50g với công thức phức hợp dưỡng trắng Vitamin B3 giúp tăng cường khả năng dưỡng trắng từ sâu bên trong, làm mờ vết thâm sạm, cho làn da trắng hồng rạng rỡ bên ngoài.",
        "Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond's White Beauty 50g với công thức phức hợp dưỡng trắng Vitamin B3+ giúp tăng cường khả năng dưỡng trắng da liên tục suốt cả đêm từ sâu bên trong, mang đến làn da sáng mịn rạng rỡ sau mỗi lần sử dụng.",
        "Kem Dưỡng Trắng Da Ban Đêm Senka White Beauty Glow Gel Cream 50g Với công nghệ độc quyền và tiên tiến Natu-Ence từ Shiseido giúp nâng cao hiệu quả của các dưỡng chất từ thiên nhiên, Kem dưỡng trắng da ban đêm Senka sẽ bổ sung các dưỡng chất cần thiết và độ ẩm cho làn da luôn mịn màng, dưỡng trắng da vượt trội.",
        "Kem dưỡng nâng tông da trắng hồng tự nhiên Laneige White Dew Tone Up Cream 50ml giúp làm sáng ngay lập tức vùng da bị vàng và xỉn màu với kết cấu giống dạng sữa dưỡng giàu độ ẩm. Công thức sữa dưỡng dầu/nước chứa TiO2 (titanium dioxide) mang đến hiệu quả làm tăng tông màu da cho những vùng da bị tối hay xỉn màu",
        "Kem Dưỡng Nivea Ngọc Trai Làm Sáng Da Ban Đêm 50ml 5 in 1 Pearl Filler Pearl White Night Face Cream Công thức vượt trội được chiết xuất từ các dưỡng chất thiên nhiên, vừa giúp dưỡng trắng da, ngay cả làn da sạm đen, vừa nuôi dưỡng làn da trong lúc ngủ. Cho bạn làn da trắng mượt như nước, không ngại bắt nắng",
        "Phấn Phủ Dưỡng Ẩm Dạng Nén Laneige Light Fit Pact 9.5g. Kết cấu mỏng nhẹ, độ bám dính cao Giúp che phủ hiệu quả hoàn hảo Thể hiện màu sắc tự nhiên",
        "Phấn Phủ Kiềm Dầu Dạng Bột Khoáng Innisfree No Sebum Mineral Powder 5g dạng bột có chiết xuất 100% từ bạc hà và hạt ngọc trai giúp kiềm hút dầu và loại bỏ bã nhờn dư thừa trên da mang đến làn da sáng mịn.", 
        "Sữa Rửa Mặt Ngăn Ngừa Lão Hóa Pond's Age Miracle 100g gồm 6 dưỡng chất chuyên biệt, giúp thúc đẩy quá trình tái tạo da đẩy lùi quá trình lão hóa.", 
        "Sữa Rửa Mặt Ngăn Ngừa Lão Hóa Da Innisfree Jeju Pomegranate Revitalizing Foam Cleanser 150ml từ nước ép lựu và dầu hạt lựu giúp làm sạch da sâu da mà không gây khô căng."
        ]

In [3]:
df = pd.DataFrame(data, columns= ['product'])

In [4]:
df.head()

,product
0,Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond'...
1,Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond'...
2,Kem Dưỡng Trắng Da Ban Đêm Senka White Beauty ...
3,Kem dưỡng nâng tông da trắng hồng tự nhiên Lan...
4,Kem Dưỡng Nivea Ngọc Trai Làm Sáng Da Ban Đêm ...


In [5]:
# word_tokenize de tach tu
df['product_wt'] = df['product'].apply(lambda x: word_tokenize(x, format = 'text'))

In [6]:
df.head(2)

,product,product_wt
0,Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond'...,Kem Dưỡng Da Ban_Ngày Trắng_Hồng Rạng_Rỡ Pond'...
1,Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond'...,Kem Dưỡng Da Đêm_Ngày Trắng_Hồng Rạng_Rỡ Pond'...


In [7]:
# postag de nhan dang tu , tinh tu , dong tu...
products_postag = df['product'].apply(lambda x: pos_tag(x))
products_postag

0    [(Kem, Np), (Dưỡng, Np), (Da, Np), (Ban Ngày, ...
1    [(Kem, Np), (Dưỡng, Np), (Da, Np), (Đêm Ngày, ...
2    [(Kem, Np), (Dưỡng, Np), (Trắng Da, Np), (Ban ...
3    [(Kem, N), (dưỡng, N), (nâng, V), (tông, N), (...
4    [(Kem, Np), (Dưỡng, Np), (Nivea, Np), (Ngọc Tr...
5    [(Phấn, Np), (Phủ, Np), (Dưỡng, Np), (Ẩm, Np),...
6    [(Phấn, Np), (Phủ, Np), (Kiềm, Np), (Dầu, Np),...
7    [(Sữa, N), (Rửa, Np), (Mặt, Np), (Ngăn Ngừa, N...
8    [(Sữa, N), (Rửa, Np), (Mặt, Np), (Ngăn Ngừa, N...
Name: product, dtype: object

In [8]:
# sent_tokennize : de tao ra 1 cau hoan chinh
products_sent = df['product'].apply(lambda x: sent_tokenize(x))
products_sent

0    [Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond...
1    [Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond...
2    [Kem Dưỡng Trắng Da Ban Đêm Senka White Beauty...
3    [Kem dưỡng nâng tông da trắng hồng tự nhiên La...
4    [Kem Dưỡng Nivea Ngọc Trai Làm Sáng Da Ban Đêm...
5    [Phấn Phủ Dưỡng Ẩm Dạng Nén Laneige Light Fit ...
6    [Phấn Phủ Kiềm Dầu Dạng Bột Khoáng Innisfree N...
7    [Sữa Rửa Mặt Ngăn Ngừa Lão Hóa Pond's Age Mira...
8    [Sữa Rửa Mặt Ngăn Ngừa Lão Hóa Da Innisfree Je...
Name: product, dtype: object

## Gensim

In [9]:
# tokenize(split) the sentences into words : lay cac cau thanh tu
products_gem  = [[text for text in x.split()] for x in df.product_wt]

In [10]:
products_gem[:1]

[['Kem',
  'Dưỡng',
  'Da',
  'Ban_Ngày',
  'Trắng_Hồng',
  'Rạng_Rỡ',
  "Pond's_White_Beauty",
  '50',
  'g',
  'với',
  'công_thức',
  'phức_hợp',
  'dưỡng',
  'trắng',
  'Vitamin_B3',
  'giúp',
  'tăng_cường',
  'khả_năng',
  'dưỡng',
  'trắng',
  'từ',
  'sâu',
  'bên',
  'trong',
  ',',
  'làm',
  'mờ',
  'vết',
  'thâm',
  'sạm',
  ',',
  'cho',
  'làn',
  'da',
  'trắng',
  'hồng',
  'rạng_rỡ',
  'bên',
  'ngoài',
  '.']]

In [11]:
# su dung regular expression de xoa ky tu dac biet
import re

In [12]:
# stop_words = open('vietnamese-stopwords.txt' , 'r', encoding='utf8')
# stop_words.read().split('\n')
with open('vietnamese-stopwords.txt' , 'r', encoding='utf8') as stop_words:
    stop_words = stop_words.read().split('\n')

In [13]:
stop_words

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy',
 'ba',
 'ba_ba',
 'ba_bản',
 'ba_cùng',
 'ba_họ',
 'ba_ngày',
 'ba_ngôi',
 'ba_tăng',
 'bao_giờ',
 'bao_lâu',
 'bao_nhiêu',
 'bao_nả',
 'bay_biến',
 'biết',
 'biết_bao',
 'biết_bao_nhiêu',
 'biết_chắc',
 'biết_chừng_nào',
 'biết_mình',
 'biết_mấy',
 'biết_thế',
 'biết_trước',
 'biết_việc',
 'biết_đâu',
 'biết_đâu_chừng',
 'biết_đâu_đấy',
 'biết_được',
 'buổi',
 'buổi_làm',
 'buổi_mới',
 'buổi_ngày',
 'buổi_sớm',
 'bà',
 'bà_ấy',
 'bài',
 'bài_bác',
 'bài_bỏ',
 'bài_cái',
 'bác',
 'bán',
 'bán_cấp',
 'bán_dạ',
 'bán_thế',
 'bây_bẩy',
 'bây_chừ',
 'bây_giờ',
 'bây_nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên_bị',
 'bên_có',
 'bên_cạnh',
 'bông',
 'bước',
 'bước_khỏi',
 'bước_tới',
 'bước_đi',
 'bạn',
 'bản',
 'bản_bộ',
 'bản_riêng',
 'bản_thân',
 'bản_ý',
 'bất_chợt',
 'bất_cứ',
 'bất_giác',
 'bất_kì',
 'bất_kể',
 'bất_kỳ',
 'bất_luận',
 'bất_ngờ',
 'bất_nhược',
 'bất_quá',
 'bất_quá_chỉ',
 'bất_thình_l

In [14]:
# remove some special elements in texts
def ham_xu_ly(stop_words , products_gem):
    products_gem_re = [[re.sub('[0-9]+' , '', e) for e in text] for text in products_gem] # so
    products_gem_re = [[t.lower() for t in text if not t in['', ' ', ',', '.', '...', '-',':',';','?', '%','(', ')','+', '/', 'g','ml']]for text in products_gem_re] # ky tu dac biet
    products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem_re]  # stopword
    return products_gem_re

In [15]:
products_gem_re = ham_xu_ly(stop_words, products_gem)

In [16]:
# su dung gensim de thiet lap list tao dict cho moi tu duy nhat
dictionary = corpora.Dictionary(products_gem_re)

In [17]:
# List of features in dict
dictionary.token2id

{'ban_ngày': 0,
 'công_thức': 1,
 'da': 2,
 'dưỡng': 3,
 'giúp': 4,
 'hồng': 5,
 'kem': 6,
 'khả_năng': 7,
 'làn': 8,
 'mờ': 9,
 'phức_hợp': 10,
 "pond's_white_beauty": 11,
 'rạng_rỡ': 12,
 'sâu': 13,
 'sạm': 14,
 'thâm': 15,
 'trắng': 16,
 'trắng_hồng': 17,
 'tăng_cường': 18,
 'vitamin_b': 19,
 'vết': 20,
 'liên_tục': 21,
 'mịn': 22,
 'suốt': 23,
 'vitamin_b_+': 24,
 'đêm': 25,
 'đêm_ngày': 26,
 'ban_đêm': 27,
 'bổ_sung': 28,
 'chất': 29,
 'công_nghệ': 30,
 'cần_thiết': 31,
 'hiệu_quả': 32,
 'kem_dưỡng': 33,
 'mịn_màng': 34,
 'natu-ence': 35,
 'nâng': 36,
 'senka': 37,
 'senka_white_beauty_glow_gel_cream': 38,
 'shiseido': 39,
 'thiên_nhiên': 40,
 'tiên_tiến': 41,
 'trắng_da': 42,
 'vượt_trội': 43,
 'độ_ẩm': 44,
 'độc_quyền': 45,
 'chứa': 46,
 'dioxide': 47,
 'dạng': 48,
 'dầu': 49,
 'giàu': 50,
 'kết_cấu': 51,
 'laneige_white_dew_tone_up_cream_': 52,
 'màu': 53,
 'màu_da': 54,
 'sữa': 55,
 'tio': 56,
 'titanium': 57,
 'tông': 58,
 'tối': 59,
 'tự_nhiên': 60,
 'vàng': 61,
 'xỉn': 62,


In [18]:
len(dictionary)

123

In [19]:
# de so duy nhat
feature_cnt = len(dictionary.token2id)

In [20]:
feature_cnt

123

In [21]:
# co duoc xac chet dua tren dict
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [22]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 2),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1)]

In [23]:
# Use TF-IDF Model to process corpus , obtaining index
tfidf = models.TfidfModel(corpus)
# tinh toan su tung tu trong matan thua thot
index  = similarities.SparseMatrixSimilarity(tfidf[corpus],
    num_features= feature_cnt)

In [24]:
# when user choose 1 product
# gia su la chon san pham dau tine de xem, index =0
products_selection = df.head(1)

In [25]:
products_selection

,product,product_wt
0,Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond'...,Kem Dưỡng Da Ban_Ngày Trắng_Hồng Rạng_Rỡ Pond'...


In [26]:
type(products_selection['product_wt'])

pandas.core.series.Series

In [27]:
# san pham dang xem
name_description_pre = products_selection['product_wt'].to_string(index =False)

In [28]:
view_product = name_description_pre.lower().split()

In [29]:
# convert search word into Sparse Vectors
kw_vector = dictionary.doc2bow(view_product)

In [30]:
kw_vector

[(0, 1), (2, 1), (3, 1), (6, 1), (12, 1), (17, 1)]

In [31]:
# similarity calculation
sim = index[tfidf[kw_vector]]

In [32]:
# print result
for i in range(len(sim)):
    # vi lay mau dau tien de xem nen bo qua mau dau tien
    if i != 0:
        print('keyword is similar to doc_index %d: %.2f' %(i , sim[i]))

keyword is similar to doc_index 1: 0.34
keyword is similar to doc_index 2: 0.04
keyword is similar to doc_index 3: 0.03
keyword is similar to doc_index 4: 0.04
keyword is similar to doc_index 5: 0.01
keyword is similar to doc_index 6: 0.00
keyword is similar to doc_index 7: 0.00
keyword is similar to doc_index 8: 0.00


In [33]:
# gia su sp lien quan nhat co index la 1 nhu tren
product_famillier = df.iloc[[1]]

In [34]:
product_famillier

,product,product_wt
1,Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond'...,Kem Dưỡng Da Đêm_Ngày Trắng_Hồng Rạng_Rỡ Pond'...


## Cosine similarity

In [35]:
from numpy import dot
from numpy.linalg import norm
X = [1,2]
Y = [2,2]
cos_sim = dot(X,Y) / (norm(X) * norm(Y))
print(cos_sim)

0.9486832980505138


In [36]:
tf = TfidfVectorizer(analyzer ='word', min_df = 0, stop_words = stop_words)

In [37]:
tfidf_matrix = tf.fit_transform(df.product_wt)

In [38]:
cosine_similarities = cosine_similarity(tfidf_matrix , tfidf_matrix)

In [39]:
cosine_similarities

array([[1.        , 0.70169906, 0.29553487, 0.2240153 , 0.37679983,
        0.05058612, 0.04694264, 0.02966145, 0.07008143],
       [0.70169906, 1.        , 0.30079247, 0.21273896, 0.36985192,
        0.05240071, 0.09208238, 0.04162698, 0.09115257],
       [0.29553487, 0.30079247, 1.        , 0.24948605, 0.45583521,
        0.07990671, 0.05220335, 0.08855173, 0.05228176],
       [0.2240153 , 0.21273896, 0.24948605, 1.        , 0.23536183,
        0.14652592, 0.10595731, 0.08286884, 0.13519131],
       [0.37679983, 0.36985192, 0.45583521, 0.23536183, 1.        ,
        0.04502876, 0.15231742, 0.08266145, 0.14192056],
       [0.05058612, 0.05240071, 0.07990671, 0.14652592, 0.04502876,
        1.        , 0.12016852, 0.00997496, 0.00848998],
       [0.04694264, 0.09208238, 0.05220335, 0.10595731, 0.15231742,
        0.12016852, 1.        , 0.02544549, 0.11376423],
       [0.02966145, 0.04162698, 0.08855173, 0.08286884, 0.08266145,
        0.00997496, 0.02544549, 1.        , 0.25026857],


In [40]:
cosine_similarities.shape

(9, 9)

In [41]:
# voi moi sp , lay 3 san pham tuong quan nhat
def result( df , cosine_similarities, number):
    results = {}

    for idx ,row in df.iterrows():
        similar_indices = cosine_similarities[idx].argsort()[-4:-1] # chon ra index lien quan nhat
        similar_items = [(cosine_similarities[idx][i]) for i in similar_indices] # lay ra gia tri lon nhat
        similar_items = [(cosine_similarities[idx][i] , df.index[i]) for i in similar_indices] # tao thanh 1 list cac tuple
        # print(similar_items[0:])
        results[idx] = similar_items[0:]
    return(results[number])

In [42]:
# chon vi tri so 5
result(df , cosine_similarities , 5)

[(0.07990671089121765, 2), (0.12016852131161818, 6), (0.1465259199935134, 3)]

# chon vi tri so 2, 6, 3 de gioi thieu
